## Single membership attack on TGAN

## To do:
<ul>
  <li>Data pipeline</li>
  <li>Adversary network</li>
  <li>Loss</li>
  <li>Updater</li>
  <li>Read in Generator</li>
</ul>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
src_path = 'C:/WorkDir/hecktor_train_v3/hecktor-main/src/'
sys.path.append(src_path)
import os
import pathlib
import numpy as np
import nibabel as nib
import sys
from data import utils
from nibabel.testing import data_path
import ImageViewer3D as I3D

import argparse
import glob
import imp
import subprocess
import yaml

import seaborn as sns
import pandas as pd

import chainer
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, FunctionNode, gradient_check, report, training, utils, Variable
from chainer import datasets, initializers, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
from chainer.training import extensions
import chainer.functions as F
import chainer.links as L
import cv2 as cv
from chainer import Variable
from chainer import serializers
import cupy

src_path = 'C:/WorkDir/tgan-master/tgan-master/'
sys.path.append(src_path)
import infer
import math
from scipy.ndimage import zoom

os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Chainer training

In [2]:
class MLP(Chain):

    def __init__(self,bottom_width = 4, conv_ch = 512):
        super(MLP, self).__init__()
        with self.init_scope():
            
            self.l1 = L.Linear(64*64*32,1024,initialW = chainer.initializers.Uniform(0.01))
            self.l2 = L.Linear(1024,100,initialW = chainer.initializers.Uniform(0.01))
            
            self.dc0 = L.DeconvolutionND(1, 100, 1024, 1, 1, 0, initialW=fsgen_weights['dc0/W'],initial_bias=fsgen_weights['dc0/b'])
            self.dc1 = L.DeconvolutionND(1, 1024, 512, 4, 2, 1, initialW=fsgen_weights['dc1/W'],initial_bias=fsgen_weights['dc1/b'])
            self.dc2 = L.DeconvolutionND(1, 512, 256, 4, 2, 1, initialW=fsgen_weights['dc2/W'],initial_bias=fsgen_weights['dc2/b'])
            self.dc3 = L.DeconvolutionND(1, 256, 128, 4, 2, 1, initialW=fsgen_weights['dc3/W'],initial_bias=fsgen_weights['dc3/b'])
            self.dc4 = L.DeconvolutionND(1, 128, 128, 4, 2, 1, initialW=fsgen_weights['dc4/W'],initial_bias=fsgen_weights['dc4/b'])
            self.dc5 = L.DeconvolutionND(1, 128, 100, 4, 2, 1, initialW=fsgen_weights['dc5/W'],initial_bias=fsgen_weights['dc5/b'])
            self.bn0 = L.BatchNormalization(1024, use_beta=False,initial_gamma=fsgen_weights['bn0/gamma'], initial_avg_mean=fsgen_weights['bn0/avg_mean'], initial_avg_var=fsgen_weights['bn0/avg_var'])
            self.bn1 = L.BatchNormalization(512, use_beta=False,initial_gamma=fsgen_weights['bn1/gamma'], initial_avg_mean=fsgen_weights['bn1/avg_mean'], initial_avg_var=fsgen_weights['bn1/avg_var'])
            self.bn2 = L.BatchNormalization(256, use_beta=False,initial_gamma=fsgen_weights['bn2/gamma'], initial_avg_mean=fsgen_weights['bn2/avg_mean'], initial_avg_var=fsgen_weights['bn2/avg_var'])
            self.bn3 = L.BatchNormalization(128, use_beta=False,initial_gamma=fsgen_weights['bn3/gamma'], initial_avg_mean=fsgen_weights['bn3/avg_mean'], initial_avg_var=fsgen_weights['bn3/avg_var'])
            self.bn4 = L.BatchNormalization(128, use_beta=False,initial_gamma=fsgen_weights['bn4/gamma'], initial_avg_mean=fsgen_weights['bn4/avg_mean'], initial_avg_var=fsgen_weights['bn4/avg_var'])
            
            slow_mid_dim = bottom_width * bottom_width * conv_ch // 2
            fast_mid_dim = slow_mid_dim
            
            self.l0s = L.Linear(100, slow_mid_dim, initialW=vgen_weights['l0s/W'], nobias=True)
            self.l0f = L.Linear(100, fast_mid_dim, initialW=vgen_weights['l0f/W'], nobias=True)
            self.dcg1 = L.Deconvolution2D(conv_ch, conv_ch // 2, 4, 2, 1, initialW=vgen_weights['dc1/W'], nobias=True)
            self.dcg2 = L.Deconvolution2D(conv_ch // 2, conv_ch // 4, 4, 2, 1, initialW=vgen_weights['dc2/W'], nobias=True)
            self.dcg3 = L.Deconvolution2D(conv_ch // 4, conv_ch // 8, 4, 2, 1, initialW=vgen_weights['dc3/W'], nobias=True)
            self.dcg4 = L.Deconvolution2D(conv_ch // 8, conv_ch // 16, 4, 2, 1, initialW=vgen_weights['dc4/W'], nobias=True)
            self.dcg5 = L.Deconvolution2D(conv_ch // 16, 1, 3, 1, 1, initialW=vgen_weights['dc5/W'], initial_bias=vgen_weights['dc5/b'])
            self.bn0s = L.BatchNormalization(slow_mid_dim,initial_gamma=vgen_weights['bn0s/gamma'], initial_beta=vgen_weights['bn0s/beta'], initial_avg_mean=vgen_weights['bn0s/avg_mean'], initial_avg_var=vgen_weights['bn0s/avg_var'])
            self.bn0f = L.BatchNormalization(fast_mid_dim,initial_gamma=vgen_weights['bn0s/gamma'], initial_beta=vgen_weights['bn0f/beta'], initial_avg_mean=vgen_weights['bn0f/avg_mean'], initial_avg_var=vgen_weights['bn0f/avg_var'])
            self.bng1 = L.BatchNormalization(conv_ch // 2,initial_gamma=vgen_weights['bn1/gamma'], initial_beta=vgen_weights['bn1/beta'], initial_avg_mean=vgen_weights['bn1/avg_mean'], initial_avg_var=vgen_weights['bn1/avg_var'])
            self.bng2 = L.BatchNormalization(conv_ch // 4,initial_gamma=vgen_weights['bn2/gamma'], initial_beta=vgen_weights['bn2/beta'], initial_avg_mean=vgen_weights['bn2/avg_mean'], initial_avg_var=vgen_weights['bn2/avg_var'])
            self.bng3 = L.BatchNormalization(conv_ch // 8,initial_gamma=vgen_weights['bn3/gamma'], initial_beta=vgen_weights['bn3/beta'], initial_avg_mean=vgen_weights['bn3/avg_mean'], initial_avg_var=vgen_weights['bn3/avg_var'])
            self.bng4 = L.BatchNormalization(conv_ch // 16,initial_gamma=vgen_weights['bn4/gamma'], initial_beta=vgen_weights['bn4/beta'], initial_avg_mean=vgen_weights['bn4/avg_mean'], initial_avg_var=vgen_weights['bn4/avg_var'])

    def forward(self, z_slow):
        #Adversary layers
        h = F.reshape(F.flatten(z_slow),[1,-1])
        h = F.relu(self.l1(h))
        z_slow = F.tanh(self.l2(h))
        
        with chainer.using_config('train', False):
            #Frame seed generation
            h = F.reshape(z_slow, (h.shape[0], -1, 1))
            h = F.relu(self.bn0(self.dc0(h)))
            h = F.relu(self.bn1(self.dc1(h)))
            h = F.relu(self.bn2(self.dc2(h)))
            h = F.relu(self.bn3(self.dc3(h)))
            h = F.relu(self.bn4(self.dc4(h)))
            z_fast = F.tanh(self.dc5(h))
            
            #Prep for video generation
            B, n_z_fast, n_frames = z_fast.shape
            z_fast = F.reshape(z_fast, (B * n_frames, n_z_fast)) #Reshape from (batch, frames, z_dim) to (batch*frames,z_dim)

            B, n_z_slow = z_slow.shape
            z_slow = F.reshape(F.broadcast_to(F.reshape(z_slow, (B, 1, n_z_slow)), (B, n_frames, n_z_slow)),(B * n_frames, n_z_slow))
        
            #Video generation
            n = z_slow.shape[0]
            h_slow = F.reshape(F.relu(self.bn0s(self.l0s(z_slow))), [n, 512 // 2, 4, 4])
            h_fast = F.reshape(F.relu(self.bn0f(self.l0f(z_fast))), [n, 512 // 2, 4, 4])
            h = F.concat([h_slow, h_fast], axis=1)
            h = F.relu(self.bng1(self.dcg1(h)))
            h = F.relu(self.bng2(self.dcg2(h)))
            h = F.relu(self.bng3(self.dcg3(h)))
            h = F.relu(self.bng4(self.dcg4(h)))
            x = F.tanh(self.dcg5(h))
            
        return x

gpu_id = 0  # Set to -1 if you use CPU

# model = MLP()
# if gpu_id >= 0:
#     model.to_gpu(gpu_id)

In [3]:
max_epoch = 500
fsgen_weights = np.load('C:/WorkDir/tgan-master/tgan-master/results/PrivacyTest-last20witheld-unconditional/gen_iter_28000.npz')
vgen_weights = np.load('C:/WorkDir/tgan-master/tgan-master/results/PrivacyTest-last20witheld-unconditional/vgen_iter_28000.npz')
cr_imgs = np.load('C:/WorkDir/tgan-master/tgan-master/cropped_imgs.npy')
batchsize = 1
# min_loss_array = np.empty(201)

def loss_fn(x, y):
  y = F.transpose(y,[1,0,2,3])
  d = x - (y-128)/128
  return F.sum(d * d)

for i in range(0,3):
    model = MLP()
    model.to_gpu(gpu_id)
    train = np.expand_dims(cr_imgs[16:-16,i,:,:],axis=1)
    train = np.transpose(train,[1,0,2,3]) #shape is (n,z,x,y)
    train_iter = iterators.SerialIterator(train, batchsize)

    # selection of your optimizing method
    optimizer = optimizers.Adam(alpha=0.0001)

    # Give the optimizer a reference to the model
    optimizer.setup(model)

    # Get an updater that uses the Iterator and Optimizer
    updater = training.updaters.StandardUpdater(train_iter, optimizer, device=gpu_id)


    it = -1
    min_loss = 1e5
    while train_iter.epoch < max_epoch:
        it=it+1
        # ---------- The first iteration of training loop ----------
        train_batch = train_iter.next()
        cv = chainer.Variable(np.array(train_batch,dtype='float32'))
        cv.to_gpu()
    #     target = chainer.Variable(np.random.rand(1,32,64,64).astype('float32'))
    #     target.to_gpu()

        # calculate the prediction of the model
        pred = model(cv)

        # calculation of loss function, softmax_cross_entropy
        loss = loss_fn(pred, cv)
        if it%100==0:
            print('image: ',i, '\titeration:',it,'\tL:',loss.data)
    #     if min_loss > loss.data:
    #         min_loss=loss.data
        # calculate the gradients in the model
        model.cleargrads()
        loss.backward()

        # update the parameters of the model
        optimizer.update()
    # min_loss_array[i] = min_loss
    print('min loss:', min_loss)

    pred.to_cpu()
    cv.to_cpu()

    target = (np.array(cv.data,dtype='float32')[0,:,:,:]-128)/128
    I3D.ImageSliceViewer3D(np.array(pred.data,dtype='float32')[:,0,:,:]*20+20)
    I3D.ImageSliceViewer3D(target*20+20)
    diff = ((np.array(pred.data,dtype='float32')[:,0,:,:]-target)*20)
    I3D.ImageSliceViewer3D(diff*20)
    model.to_cpu()
    del model

image:  0 	iteration: 0 	L: 731.4595


C:\Users\robbe\anaconda3\lib\site-packages\chainer\functions\connection\convolution_2d.py:295: PerformanceWarning: The best algo of conv fwd might not be selected due to lack of workspace size (8388608)
  cuda.cudnn.convolution_forward(


image:  0 	iteration: 100 	L: 7.5869064
image:  0 	iteration: 200 	L: 3.628706
image:  0 	iteration: 300 	L: 2.3816
image:  0 	iteration: 400 	L: 1.8069761
min loss: 100000.0


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

...

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

image:  2 	iteration: 0 	L: 472.3579
image:  2 	iteration: 100 	L: 3.7285557
image:  2 	iteration: 200 	L: 1.7732972
image:  2 	iteration: 300 	L: 1.588881
image:  2 	iteration: 400 	L: 1.3618542
min loss: 100000.0


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [4]:

# plt.figure()
# plt.hist(min_loss_array[:180])
# plt.hist(min_loss_array[180:])

In [5]:
# from chainer.training import extensions
# trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='chainer_test')

# trainer.extend(extensions.LogReport())
# trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
# # trainer.extend(extensions.snapshot_object(model.predictor, filename='model_epoch-{.updater.epoch}'))
# #trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))
# trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'elapsed_time']))
# trainer.extend(extensions.PlotReport(['main/loss'], x_key='epoch', file_name='loss.png'))
# trainer.extend(extensions.PlotReport(['main/accuracy'], x_key='epoch', file_name='accuracy.png'))
# trainer.extend(extensions.DumpGraph('main/loss'))

In [6]:
# trainer.run()

In [7]:
# out = model(cv)
# cv.to_cpu()
# out.to_cpu()

# target = (np.array(cv.data,dtype='float32')[0,:,:,:]-128)/128
# I3D.ImageSliceViewer3D(np.array(out.data,dtype='float32')[:,0,:,:]*20+20)
# I3D.ImageSliceViewer3D(target*20+20)